In [ ]:
# Import the libraries

from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from random import randint
import time
import re
import csv

In [ ]:
# Create data storage

pages = []
review = []
date = []
stars = []

# I am using a for loop to iterate through the pages and save the date and review information in the lists

url = 'https://www.trustpilot.com/review/zaful.com'
for i in range(1000):
    pg = url + '?page=' + str(i)
    r = requests.get(pg)
    soup = BeautifulSoup(r.text, 'lxml')

    print("Status code: " + str(r.status_code))     # Ensure that we are able to download data from all pages

    # store response into "pages" list
    pages.append(r)

    # adding wait time:
    wait_time = randint(1,4)
    print("I will sleep for " + str(wait_time) + " second/s.")
    time.sleep(wait_time)
    
    for star in soup.find_all("section", {"class":"review__content"}):
        rating = star.find("div", {"class":"star-rating star-rating--medium"}).find('img').get('alt')
        stars.append(rating)
            
    for i in range(len(soup.select("div.review-content__body"))):
        date.append(soup.select("div.review-content__header")[i].get_text())
        review.append(soup.select("div.review-content__body")[i].get_text())
        

In [ ]:
# Extract the date using regex

clean_date = []
for string in date:
    clean_date.append(re.findall(r'\d{4}-\d{2}-\d{2}', string))
#clean_date

In [ ]:
# Save only the first date in a new list

final_date = []
for i in range(len(clean_date)):
    final_date.append(clean_date[i][0])

In [ ]:
# final_date

In [ ]:
# Clean extracted reviews

clean_review = []
for string in review:
    clean_review.append(string.strip())
#clean_review

In [ ]:
# Save reviews in a dataframe

zaful_reviews = pd.DataFrame({"date": final_date,
                             "review": clean_review,
                              "rating": stars})

In [ ]:
zaful_reviews

In [ ]:
# Drop duplicates

zaful_reviews = zaful_reviews.drop_duplicates(subset = 'review', keep = 'first')

In [ ]:
# Reset index

zaful_reviews = zaful_reviews.reset_index(drop = True)

In [ ]:
# Save dataframe as a csv file

zaful_reviews.to_csv('zaful_reviews_9_dec_stars.csv')

In [ ]:
# Read csv file

zaful_reviews = pd.read_csv('zaful_reviews_9_dec_stars.csv')

In [ ]:
# Remove whitespces from the reviews using regex

zaful_reviews = zaful_reviews.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value = ["",""], regex = True)

In [ ]:
# Save to csv again

zaful_reviews.to_csv('zaful-reviews-with-rating.csv')